**Reference Link:** [RAG Systems Essentials (Analytics Vidhya)](https://courses.analyticsvidhya.com/courses/take/rag-systems-essentials/lessons/60148017-hands-on-deep-dive-into-rag-evaluation-metrics-generator-metrics-i)

# Exploring Embedding Models in LangChain

## Install OpenAI, HuggingFace and LangChain dependencies

In [0]:
!pip install langchain==0.3.11
!pip install langchain-openai==0.2.12
!pip install langchain-community==0.3.11
!pip install langchain-huggingface==0.1.2

## Enter Open AI and HuggingFace API Tokens

In [0]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

In [0]:
from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass('Enter HuggingFace Auth Token Key: ')

## Setup Environment Variables

In [0]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HUGGINGFACEHUB_API_TOKEN

## Embedding models

The Embeddings class is a class designed for interfacing with text embedding models. There are lots of embedding model providers (OpenAI, Cohere, Hugging Face, etc) - this class is designed to provide a standard interface for all of them.

Embeddings create a vector representation of a piece of text. This is useful because it means we can think about text in the vector space, and do things like semantic search where we look for pieces of text that are most similar in the vector space.

The base Embeddings class in LangChain provides two methods: one for embedding documents and one for embedding a query. The former takes as input multiple texts, while the latter takes a single text. The reason for having these as two separate methods is that some embedding providers have different embedding methods for documents (to be searched over) vs queries (the search query itself).

In [0]:
docs = [
    "cats eat and sleep",
    "dogs eat and bark",
    "cars drive fast",
    "vehicles include trucks and cars"
]

Embeddings create a vector representation of a piece of text. This is useful because it means we can think about text in the vector space, and do things like semantic search where we look for pieces of text that are most similar in the vector space.

The base Embeddings class in LangChain provides two methods: one for embedding documents and one for embedding a query. The former, `.embed_documents`, takes as input multiple texts, while the latter, `.embed_query`, takes a single text. The reason for having these as two separate methods is that some embedding providers have different embedding methods for documents (to be searched over) vs queries (the search query itself).

- `.embed_query`  will return a list of floats,
- `.embed_documents` returns a list of lists of floats.

### Open AI Embedding Models

LangChain enables us to access Open AI embedding models which include the newest models: a smaller and highly efficient `text-embedding-3-small` model, and a larger and more powerful `text-embedding-3-large` model.

In [0]:
from langchain_openai import OpenAIEmbeddings

# details here: https://openai.com/blog/new-embedding-models-and-api-updates
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

In [0]:
embeddings = openai_embed_model.embed_documents(docs)

In [0]:
len(embeddings)

In [0]:
print(embeddings)

In [0]:
len(embeddings[0])

In [0]:
print(embeddings[0])

In [0]:
docs

In [0]:
from sklearn.metrics.pairwise import cosine_similarity

sim_matrix = cosine_similarity(embeddings)
sim_matrix

## Open Source Embedding Models on HuggingFace

`langchain-huggingface` integrates seamlessly with LangChain, providing an efficient and effective way to utilize Hugging Face models within the LangChain ecosystem.

`HuggingFaceEmbeddings`uses `sentence-transformers` embeddings. It computes the embedding locally, using your computer resources and allows you to access open or open source embedding LLMs hosted on HuggingFace.

In [0]:
from langchain_huggingface import HuggingFaceEmbeddings

# check out model details here: https://huggingface.co/mixedbread-ai/mxbai-embed-large-v1
model_name = "mixedbread-ai/mxbai-embed-large-v1"

hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
)

In [0]:
embeddings = hf_embeddings.embed_documents(docs)

In [0]:
len(embeddings)

In [0]:
len(embeddings[0])

In [0]:
docs

In [0]:
sim_matrix = cosine_similarity(embeddings)
sim_matrix

## Build a small search engine!

### Load Knowledgebase documents

In [0]:
documents = [
    'Quantum mechanics describes the behavior of very small particles.',
    'Photosynthesis is the process by which green plants make food using sunlight.',
    "Shakespeare's plays are a testament to English literature.",
    'Artificial Intelligence aims to create machines that can think and learn.',
    'The pyramids of Egypt are historical monuments that have stood for thousands of years.',
    'Biology is the study of living organisms and their interactions with the environment.',
    'Music therapy can aid in the mental well-being of individuals.',
    'The Milky Way is just one of billions of galaxies in the universe.',
    'Economic theories help understand the distribution of resources in society.',
    'Yoga is an ancient practice that involves physical postures and meditation.'
]

In [0]:
len(documents)

### Get document embeddings

In [0]:
document_embeddings = openai_embed_model.embed_documents(documents)

### Let's try to find the most similar document for one query

In [0]:
new_text = 'What is AI?'
new_text

In [0]:
query_embedding = openai_embed_model.embed_query(new_text)

In [0]:
cosine_similarities = cosine_similarity([query_embedding], document_embeddings)
cosine_similarities

In [0]:
import numpy as np

documents[np.argmax(cosine_similarities[0])]

### Create Search Engine function

In [0]:
def semantic_search_engine(query, embedder_model):
  query_embedding = embedder_model.embed_query(query)
  cos_scores = cosine_similarity([query_embedding], document_embeddings)[0]
  top_result_id = np.argmax(cos_scores)
  return documents[top_result_id]

### Try out the function

In [0]:
new_sentence = 'Tell me about AI'
semantic_search_engine(new_sentence, openai_embed_model)

In [0]:
new_sentence = 'Do you know about the pyramids?'
semantic_search_engine(new_sentence, openai_embed_model)

In [0]:
new_sentence = 'How do plants survive?'
semantic_search_engine(new_sentence, openai_embed_model)